In [58]:
# Normal Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os
# Machine Learning Packages
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
# Torch Packages
import torch
import torch.nn as nn
import torch.optim as optim

# 0. Dataset Overview
> Reference: https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data

**Dataset**: Credit Score Dataset

**Task**: Given a person’s credit-related information, build a machine learning model that can classify the credit score.

**Columns**:
- `id`:	Unique identifier for each record.
- `customer_id`:	Unique identifier for each customer.
- `month`:	Month of the transaction or record.
- `name`:	Customer’s name.
- `age`:	The customer’s age.
- `ssn`:	Customer’s social security number.
- `occupation`:	The customer’s occupation.
- `annual_income`:	The customer’s annual income.
- `monthly_inhand_salary`:	The customer’s monthly take-home salary.
- `num_bank_accounts`:	Total number of bank accounts owned by the customer.
- `num_credit_card`:	Total number of credit cards held by the customer.
- `interest_rate`:	The interest rate applied to loans or credits.
- `num_of_loan`:	Number of loans the customer has taken.
- `type_of_loan`:	Categories of loans obtained by the customer.
- `delay_from_due_date`:	The delay in payment relative to the due date.
- `num_of_delayed_payment`:	Total instances of late payments made by the customer.
- `changed_credit_limit`:	Adjustments made to the customer’s credit limit.
- `num_credit_inquiries`:	Number of inquiries made regarding the customer's credit.
- `credit_mix`:	The variety of credit types the customer uses (e.g., loans, credit cards).
- `outstanding_debt`:	Total amount of debt the customer currently owes.
- `credit_utilization_ratio`:	Proportion of credit used compared to the total credit limit.
- `credit_history_age`:	Duration of the customer’s credit history.
- `payment_of_min_amount`:	Indicates if the customer pays the minimum required amount each month.
- `total_emi_per_month`:	Total Equated Monthly Installment (EMI) paid by the customer.
- `amount_invested_monthly`:	Monthly investment amount made by the customer.
- `payment_behaviour`:	Customer’s payment habits and tendencies.
- `monthly_balance`:	The remaining balance in the customer’s account at the end of each month.
- `credit_score`:	The customer’s credit score (target variable: "Good," "Poor," "Standard").

**Initial thoughts**:
- Privacy concerns: `ssn`, `name` are sensitive data, we should not include them in the final model.
- Usefulness: `id`, `customer_id` are not useful for prediction, we should not include them in the final model.
- Time series: `month` is a time series data, but it does not contain year information, we should be careful when using it.
- Imbalance: `credit_score` is an imbalanced data, we should be careful when using it.
- Loan Type: `type_of_loan` may contain multiple types of loans, we should be careful when using it.

In [59]:
# Data loading
data_path = 'data/'
train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')

# Column Processing (lower case)
train_df.columns = [col.lower() for col in train_df.columns]
test_df.columns = [col.lower() for col in test_df.columns]

# Drop ID column
train_df = train_df.drop(['id','name', 'ssn'], axis=1)
test_df = test_df.drop(['id','name', 'ssn'], axis=1)

# Data overview
print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Prediction target: {train_df['credit_score'].value_counts()}")

Train data shape: (100000, 25)
Test data shape: (50000, 24)
Prediction target: credit_score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64


/tmp/ipykernel_245467/1528034292.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(data_path + 'train.csv')


In [60]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customer_id               100000 non-null  object 
 1   month                     100000 non-null  object 
 2   age                       100000 non-null  object 
 3   occupation                100000 non-null  object 
 4   annual_income             100000 non-null  object 
 5   monthly_inhand_salary     84998 non-null   float64
 6   num_bank_accounts         100000 non-null  int64  
 7   num_credit_card           100000 non-null  int64  
 8   interest_rate             100000 non-null  int64  
 9   num_of_loan               100000 non-null  object 
 10  type_of_loan              88592 non-null   object 
 11  delay_from_due_date       100000 non-null  int64  
 12  num_of_delayed_payment    92998 non-null   object 
 13  changed_credit_limit      100000 non-null  ob

In [61]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               50000 non-null  object 
 1   month                     50000 non-null  object 
 2   age                       50000 non-null  object 
 3   occupation                50000 non-null  object 
 4   annual_income             50000 non-null  object 
 5   monthly_inhand_salary     42502 non-null  float64
 6   num_bank_accounts         50000 non-null  int64  
 7   num_credit_card           50000 non-null  int64  
 8   interest_rate             50000 non-null  int64  
 9   num_of_loan               50000 non-null  object 
 10  type_of_loan              44296 non-null  object 
 11  delay_from_due_date       50000 non-null  int64  
 12  num_of_delayed_payment    46502 non-null  object 
 13  changed_credit_limit      50000 non-null  object 
 14  num_cr

# 1. Exploratory Data Analysis
> EDA is the process of visualizing and summarizing data in order to gain insights and make informed decisions. During the EDA, we will clean the data, visualize the data, and summarize the data.

## 1.1. Data Cleaning
For this dataset, we will not perform the the normal data cleaning process since weird values are present. It is necessary to mark the weird values and convert them to missing value, and handle them in the final stage.
## 1.1.1. Consistency Check

In [62]:
# Unique values in selected columns
total_df = pd.concat([train_df, test_df])
train_df_index = len(train_df)
selected_columns = [col for col in train_df.columns if col not in ['customer_id', 'credit_score']]
for col in selected_columns:
    print(f"Unique values for `{col}` : {total_df[col].unique()}")
    print(total_df[col].value_counts())
    print()

Unique values for `month` : ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']
month
January      12500
February     12500
March        12500
April        12500
May          12500
June         12500
July         12500
August       12500
September    12500
October      12500
November     12500
December     12500
Name: count, dtype: int64

Unique values for `age` : ['23' '-500' '28_' ... '326' '7910' '5826']
age
39      4198
32      4189
28      4173
26      4140
35      4130
        ... 
6427       1
5924       1
8386       1
529        1
7694       1
Name: count, Length: 2524, dtype: int64

Unique values for `occupation` : ['Scientist' '_______' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']
occupation
_______          10500
Lawyer            9899
Engineer          9562
Architect         9550
Mechanic         

In [63]:
# Check first 5 values in each column that are not numeric data
for col in selected_columns:
    print(f"{col} : {[value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()][:5]}")

month : ['January', 'February', 'March', 'April', 'May']
age : ['-500', '28_', '34_', '30_', '24_']
occupation : ['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur']
annual_income : ['34847.84_', '30689.89_', '35547.71_', '34081.38_', '114838.41_']
monthly_inhand_salary : [np.float64(nan)]


num_bank_accounts : [np.int64(-1)]
num_credit_card : []
interest_rate : []
num_of_loan : ['-100', '0_', '3_', '2_', '5_']
type_of_loan : ['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan', 'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified', 'Not Specified', nan]
delay_from_due_date : [np.int64(-1), np.int64(-2), np.int64(-3), np.int64(-5), np.int64(-4)]
num_of_delayed_payment : [nan, '8_', '-1', '3_', '2_']
changed_credit_limit : ['_', '-2.01', '-1.01', '-3.01', '-1.2400000000000002']
num_credit_inquiries : [np.float64(nan)]
credit_mix : ['_', 'Good', 'Standard', 'Bad']
outstanding_debt : ['1328.93_', '1283.37_', '2797.17_', '3818.57_', '343.84_']
credit_utilization_ratio : []
credit_history_age : ['22 Years and 1 Months', nan, '22 Years and 3 Months', '22 Years and 4 Months', '22 Years and 5 Months']
payment_of_min_amount : ['No', 'NM', 'Yes']
total_emi_per_month : []
amount_invested_monthly : ['__10000__', nan]
payment_behaviour : ['High_spent_Small_

From above observation, we could identify the following:
- Time Information: `month` has 12 values, it is categorical data, `credit_history_age` has different format - '22 Years and 1 Months', should be cleaned into months
- Irregular Format: `age`, `annual_income`, `num_of_loan`, `num_of_delayed_payment`, `outstanding_debt` and more columns might have irregular format (eg. 3.12_ / 5_), `credit_history_age` have year and month information, but the data might need to be cleaned into months (e.g. 22 Years and 1 Months)
- Irregular Values: `age`, `delay_from_due_date`, `num_of_loan` and more columns might have irregular values (eg. Negative values or large values)
- Missing/Unknown Values: Columns might have missing/unknown values, but in different formats (e.g. `_______`, `nan`)
- More Values in One Column: `type_of_loan` has more than 1 value in one column, need to be cleaned via one-hot encoding

In [64]:
# 1. Irregular Format: Correct format for unknown values - '_'or 'nan'
total_df['credit_mix'] = total_df['credit_mix'].apply(lambda x: np.nan if x == '_' else x)
total_df['changed_credit_limit'] = total_df['changed_credit_limit'].apply(lambda x: np.nan if x == '_' else x)
total_df['occupation'] = total_df['occupation'].apply(lambda x: np.nan if x.replace("_", "") == '' else x)
total_df['payment_behaviour'] = total_df['payment_behaviour'].apply(lambda x: np.nan if x == '!@9#%8' else x)
total_df['amount_invested_monthly'] = total_df['amount_invested_monthly'].apply(lambda x: np.nan if x == '__10000__' else x)

# 2. Irregular Format: Convert original data to numeric, while handling unknown values '_'
int_format_cols = ['age', 'num_of_loan', 'num_of_delayed_payment']
float_format_cols = ['annual_income', 'amount_invested_monthly', 'outstanding_debt', 'monthly_balance', 'changed_credit_limit']
for col in int_format_cols:
    total_df[col] = total_df[col].apply(lambda x: int(float(str(x).replace('_', ''))) if  x != '_' and pd.notnull(x) else np.nan)
for col in float_format_cols:
    total_df[col] = total_df[col].apply(lambda x: float(str(x).replace('_', '')) if  x != '_' and pd.notnull(x) else np.nan)

# 3. Irregular Format: Convert original data to numeric, while handling unknown values '_'
month_mapping = {
    'January': 0, 'February': 1, 'March': 2, 
    'April': 3, 'May': 4, 'June': 5, 
    'July': 6, 'August': 7, 'September': 8,
    'October': 9,'November': 10,'December': 11
}
total_df['month'] = total_df['month'].map(month_mapping)

# 3. credit_history_age year and month -> year * 12 + month
total_df['credit_history_age_in_month'] = total_df['credit_history_age'].apply(lambda x: int(float(str(x).split(' ')[0])) * 12 + int(float(str(x).split(' ')[3])) if pd.notnull(x) else np.nan)
total_df = total_df.drop(['credit_history_age'], axis=1)

# 4. One-hot encoding
total_df['type_of_loan'] = total_df['type_of_loan'].str.replace(' and ', ' ', regex=False)
total_df['type_of_loan'] = total_df['type_of_loan'].str.split(', ')
total_df['type_of_loan'] = total_df['type_of_loan'].apply(lambda x: x if isinstance(x, list) else [])

mlb = MultiLabelBinarizer()
loan_type_encoded = pd.DataFrame(mlb.fit_transform(total_df['type_of_loan']),
                                 columns=[f"{cls.replace('-', '_').replace(' ', '_').lower()}" for cls in mlb.classes_],
                                 index=total_df.index)

total_df = pd.concat([total_df, loan_type_encoded], axis=1)
total_df = total_df.drop('type_of_loan', axis=1)

In [65]:
# # Check first 5 values in each column that are not numeric data
for col in total_df.columns:
    print(f"{col} : {[value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()][:5]}")
total_df.info()

customer_id : ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb']
month : []
age : [np.int64(-500)]
occupation : ['Scientist', nan, 'Teacher', 'Engineer', 'Entrepreneur']
annual_income : []
monthly_inhand_salary : [np.float64(nan)]
num_bank_accounts : [np.int64(-1)]
num_credit_card : []
interest_rate : []
num_of_loan : [np.int64(-100)]
delay_from_due_date : [np.int64(-1), np.int64(-2), np.int64(-3), np.int64(-5), np.int64(-4)]
num_of_delayed_payment : [np.float64(nan), np.float64(-1.0), np.float64(-3.0), np.float64(-2.0)]
changed_credit_limit : [np.float64(nan), np.float64(-2.01), np.float64(-1.01), np.float64(-3.01), np.float64(-1.2400000000000002)]
num_credit_inquiries : [np.float64(nan)]
credit_mix : [nan, 'Good', 'Standard', 'Bad']
outstanding_debt : []
credit_utilization_ratio : []
payment_of_min_amount : ['No', 'NM', 'Yes']
total_emi_per_month : []
amount_invested_monthly : [np.float64(nan)]
payment_behaviour : ['High_spent_Small_value_payments', 'Low_spent_Larg

In [66]:
for col in total_df.columns:
    result = [value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()]
    if len(result) > 0:
        print(f"{col} : {result}")

customer_id : ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb', 'CUS_0x95ee', 'CUS_0x284a', 'CUS_0x5407', 'CUS_0x4157', 'CUS_0xba08', 'CUS_0xa66b', 'CUS_0xc0ab', 'CUS_0x3e45', 'CUS_0x6c66', 'CUS_0xff4', 'CUS_0x33d2', 'CUS_0x6070', 'CUS_0xfdb', 'CUS_0x3553', 'CUS_0x4100', 'CUS_0x132f', 'CUS_0xa16e', 'CUS_0xac86', 'CUS_0x5b48', 'CUS_0xa5f9', 'CUS_0x4d43', 'CUS_0xb95f', 'CUS_0x3edc', 'CUS_0xbffe', 'CUS_0xb681', 'CUS_0x9a71', 'CUS_0x6048', 'CUS_0x95b5', 'CUS_0x4004', 'CUS_0xb101', 'CUS_0x9b94', 'CUS_0x4080', 'CUS_0x706a', 'CUS_0x42ac', 'CUS_0x571f', 'CUS_0xb5ff', 'CUS_0x6a1b', 'CUS_0x9bc1', 'CUS_0xc65', 'CUS_0xaedb', 'CUS_0x8e9b', 'CUS_0x609d', 'CUS_0x75c6', 'CUS_0x9b3c', 'CUS_0x22a3', 'CUS_0xb14', 'CUS_0xb986', 'CUS_0xbe1b', 'CUS_0x9ce6', 'CUS_0x6ffb', 'CUS_0x4315', 'CUS_0x3f5b', 'CUS_0x9d78', 'CUS_0x47db', 'CUS_0x8f17', 'CUS_0x7d0b', 'CUS_0x7504', 'CUS_0x774', 'CUS_0x9f70', 'CUS_0xab76', 'CUS_0x5cdf', 'CUS_0x2184', 'CUS_0xa156', 'CUS_0x74f2', 'CUS_0x308b', 'CUS_0x2827

## 1.1.2. Outlier Check

In [67]:
numeric_cols = total_df.select_dtypes(include=np.number).columns.tolist()
for col in total_df.columns:
    if col in numeric_cols:
        print(f"{col}: {sorted(list(total_df[col].unique()), reverse=False)[:5]}")
    else:   
        print(f"{col}: {list(total_df[col].unique())[:5]}")

customer_id: ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb']
month: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
age: [np.int64(-500), np.int64(14), np.int64(15), np.int64(16), np.int64(17)]
occupation: ['Scientist', nan, 'Teacher', 'Engineer', 'Entrepreneur']
annual_income: [np.float64(7005.93), np.float64(7006.035), np.float64(7006.52), np.float64(7011.685), np.float64(7012.31)]


monthly_inhand_salary: [np.float64(303.6454166666666), np.float64(319.55625), np.float64(332.12833333333333), np.float64(333.5966666666667), np.float64(355.20833333333337)]
num_bank_accounts: [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
num_credit_card: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
interest_rate: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
num_of_loan: [np.int64(-100), np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
delay_from_due_date: [np.int64(-5), np.int64(-4), np.int64(-3), np.int64(-2), np.int64(-1)]
num_of_delayed_payment: [np.float64(7.0), np.float64(nan), np.float64(-3.0), np.float64(-2.0), np.float64(-1.0)]
changed_credit_limit: [np.float64(-6.48), np.float64(-6.3100000000000005), np.float64(-6.2), np.float64(-5.93), np.float64(-5.78)]
num_credit_inquiries: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)]
credit_mix: [nan, 'Good', 'Standard', 'Bad']
outsta

In [68]:
selected_columns = ['age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance', 
       'credit_history_age_in_month']

print(total_df[selected_columns].describe())

def detect_outlier(df, selected_columns):
    for col in selected_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        print(f"\n{col}:")
        print(f"Outlier Number: {len(outliers)}")
        if len(outliers) > 0:
            print("Outlier Example:")
            print(outliers.head())

detect_outlier(total_df, selected_columns)

                 age  annual_income  monthly_inhand_salary  num_bank_accounts  \
count  150000.000000   1.500000e+05          127500.000000      150000.000000   
mean      110.337940   1.730552e+05            4190.115139          17.006940   
std       684.066779   1.404215e+06            3180.489657         117.069476   
min      -500.000000   7.005930e+03             303.645417          -1.000000   
25%        25.000000   1.945549e+04            1625.265833           3.000000   
50%        33.000000   3.757861e+04            3091.000000           6.000000   
75%        42.000000   7.279690e+04            5948.454596           7.000000   
max      8698.000000   2.419806e+07           15204.633333        1798.000000   

       num_credit_card  interest_rate    num_of_loan  delay_from_due_date  \
count    150000.000000  150000.000000  150000.000000        150000.000000   
mean         22.623447      71.234907       3.141093            21.063400   
std         129.143006     461.537193  

We could observe some weird value in the data.
- Negative values: `age`, `num_bank_accounts`, `num_of_loan`, `num_of_delayed_payment`, `monthly_balance` should not have negative value
- Large values: 
    - `age`: normally, age should not exceed 120
    - `annual_income`: normally, annual_income should not exceed 1e7
    - `num_bank_accounts`: normally, bank account should not exceed 100
    - `interest_rate`: normally, interest_rate should not exceed 100
    - `num_of_loan`: normally, number of loan should not exceed 100
    - `num_of_delayed_payment`: normally, number of delayed payment should not exceed 100
    - ....More could been observe from describe()
- Mismatch in Age: `age` should be at least larger than `credit_history_age_in_month / 12`

Rest columns seems to be normal compared to other extremes.

In [69]:
# Negative Values
total_df['age'] = total_df['age'].apply(lambda x: x if x >= 0 and x <= 120 else np.nan)
total_df['num_bank_accounts'] = total_df['num_bank_accounts'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['num_of_loan'] = total_df['num_of_loan'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['num_of_delayed_payment'] = total_df['num_of_delayed_payment'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['monthly_balance'] = total_df['monthly_balance'].apply(lambda x: x if x >= -100000 and x <= 10000000 else np.nan)
# Large Values - no negative values
total_df['annual_income'] = total_df['annual_income'].apply(lambda x: x if x <= 10000000 else np.nan)
total_df['interest_rate'] = total_df['interest_rate'].apply(lambda x: x if x <= 100 else np.nan)
total_df['num_credit_card'] = total_df['num_credit_card'].apply(lambda x: x if x <= 100 else np.nan)
total_df['num_credit_inquiries'] = total_df['num_credit_inquiries'].apply(lambda x: x if x <= 100 else np.nan)
# Weird Values
total_df['credit_history_age_in_month'] = total_df.apply(lambda row: row['credit_history_age_in_month'] 
                                                        if row['age'] >= row['credit_history_age_in_month'] / 12 
                                                        else np.nan, axis=1)

In [70]:
selected_columns = ['month', 'age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']

print(total_df[selected_columns].describe())

def detect_outlier(df, selected_columns):
    for col in selected_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        print(f"\n{col}:")
        print(f"Outlier Number: {len(outliers)}")
        if len(outliers) > 0:
            print("Outlier Example:")
            print(outliers.head())

detect_outlier(total_df, selected_columns)

               month            age  annual_income  monthly_inhand_salary  \
count  150000.000000  145830.000000   1.491010e+05          127500.000000   
mean        5.500000      33.494686   7.069578e+04            4190.115139   
std         3.452064      10.794240   3.675344e+05            3180.489657   
min         0.000000      14.000000   7.005930e+03             303.645417   
25%         2.750000      25.000000   1.939650e+04            1625.265833   
50%         5.500000      33.000000   3.723350e+04            3091.000000   
75%         8.250000      42.000000   7.211625e+04            5948.454596   
max        11.000000     120.000000   9.976017e+06           15204.633333   

       num_bank_accounts  num_credit_card  interest_rate    num_of_loan  \
count      148116.000000    146765.000000  147025.000000  143467.000000   
mean            5.405392         5.611011      14.542942       3.551444   
std             2.996474         3.047392       8.785156       2.662183   
min   

## 1.1.3. Duplicate Check

In [71]:
# Duplicate
print(train_df[['customer_id', 'month']].duplicated().sum())
print(test_df[['customer_id', 'month']].duplicated().sum())

0
0


## 1.1.4. Missing Values
We now have two types of missing values:

1.Original Missing Values: These are the missing values that are present in the original dataset. They can be of any type, such as numerical, categorical, or text.

2.Imputed Missing Values: These are the missing values that are created by the outlier limitation process. They are created by replacing the outlier values with the upper or lower limit of the outlier detection range.

We should try to match the missing values in the original dataset with the other value in the dataset (under same customer_id). 

In [72]:
# Missing Values - Checked All Columns (Matched)
missing_df = pd.concat([
    pd.DataFrame(train_df.isnull().sum(), columns=['train_miss']),
    pd.DataFrame(test_df.isnull().sum(), columns=['test_miss']),
    pd.DataFrame(total_df.isnull().sum(), columns=['total_miss']),
], axis=1).reset_index()
missing_df[(missing_df['train_miss'] > 0) | (missing_df['test_miss'] > 0) | (missing_df['total_miss'] > 0)]

,index,train_miss,test_miss,total_miss
2,age,0.0,0.0,4170.0
3,occupation,0.0,0.0,10500.0
4,annual_income,0.0,0.0,899.0
5,monthly_inhand_salary,15002.0,7498.0,22500.0
6,num_bank_accounts,0.0,0.0,1884.0
7,num_credit_card,0.0,0.0,3235.0
8,interest_rate,0.0,0.0,2975.0
9,num_of_loan,0.0,0.0,6533.0
10,type_of_loan,11408.0,5704.0,NaN
12,num_of_delayed_payment,7002.0,3498.0,12545.0


In [73]:
# Some of missing value could be due to that customer forget to write some information
## We could check if all customers have 12 months of data, if not, it could represent forget to write / other reasons
missing_df = total_df.groupby(['customer_id']).count().reset_index()
unsatisfied_cols = []
unsatisfied_ids = set()
for col in missing_df.columns:
    if col != 'customer_id':
        unsatisfied_rows = missing_df[missing_df[col] < 12]
        if len(unsatisfied_rows) > 0:
            unsatisfied_customer_ids = set(unsatisfied_rows['customer_id'].tolist())
            unsatisfied_cols.append(col)
            unsatisfied_ids = unsatisfied_ids.union(unsatisfied_customer_ids)
print(len(unsatisfied_ids))
print(unsatisfied_cols)

12500
['age', 'occupation', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'credit_mix', 'amount_invested_monthly', 'payment_behaviour', 'monthly_balance', 'credit_score', 'credit_history_age_in_month']


In [74]:
# Check if there are columns that have only one value (and null), we can then copy the one value to null values
def find_customers_with_many_values(df, target_col, threshold=2):
    unique_values = df.groupby('customer_id')[target_col].apply(lambda x: x.unique())
    result_df = unique_values.reset_index()
    result_df.columns = ['customer_id', 'unique_values']
    result_df['num_unique'] = result_df['unique_values'].apply(len)
    filtered = result_df[result_df['num_unique'] >= threshold]
    return filtered
# Fill null values with the value that is not null for each customer
def fill_single_value_per_customer(df, target_col):
    # Find value that are not null for each customer
    single_value = (
        df[~df[target_col].isna()]
        .groupby('customer_id')[target_col]
        .agg(lambda x: x.unique()[0])  # Get the first unique value
        .rename(f'{target_col}_filled')
    )
    # Merge with original dataframe and fill null values
    df = df.merge(single_value, on='customer_id', how='left')
    df[target_col] = df[target_col].fillna(df[f'{target_col}_filled'])
    df.drop(columns=[f'{target_col}_filled'], inplace=True)
    return df

# Check and Paste columns that have only one value (and null)
one_value_cols = []
for col in unsatisfied_cols:
    filtered = find_customers_with_many_values(total_df, col)
    print(f"{col}: {filtered['num_unique'].unique()}")
    if len(filtered['num_unique'].unique()) == 1 and filtered['num_unique'].unique()[0] == 2:
        one_value_cols.append(col)

print(f"Columns with only one value (and Null Value): {one_value_cols}")
# for col in one_value_cols:
#     total_df = fill_single_value_per_customer(total_df, col)

age: [2 3 4]
occupation: [2]
annual_income: [2 3 4]
monthly_inhand_salary: [2 3]
num_bank_accounts: [2 3 4]
num_credit_card: [2 3 4]
interest_rate: [2 3]
num_of_loan: [2 3]
num_of_delayed_payment: [4 6 3 5 7 2 8]
changed_credit_limit: [3 5 4 2 6 7 8 9]
num_credit_inquiries: [2 3 4]
credit_mix: [2]
amount_invested_monthly: [12  9 11 10  8  7]
payment_behaviour: [6 5 7 4 3 2]
monthly_balance: [12 11  9 10  8  6  7]
credit_score: [3 2 4]
credit_history_age_in_month: [12 11 10  9  2  6  8  3  5  7  4]
Columns with only one value (and Null Value): ['occupation', 'credit_mix']


In [75]:
def fill_grouped_missing(df, group_col, numeric_cols, categorical_cols):
    # Numeric Columns Could Fill with Group Minimum
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col] = df.groupby(group_col)[col].transform(lambda x: x.fillna(x.min()))
    # Categorical Columns Could Fill with Group Mode
    for col in categorical_cols:
        if df[col].isnull().any():
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
    return df

# Numeric Columns Could Fill with Group Minimum
numeric_cols = ['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'amount_invested_monthly', 'monthly_balance', 'credit_history_age_in_month']
# Categorical Columns Could Fill with Group Mode
categorical_cols = ['occupation', 'credit_mix', 'payment_behaviour']
new_total_df = fill_grouped_missing(total_df, group_col='customer_id', numeric_cols = numeric_cols, categorical_cols = categorical_cols)

In [76]:
# Missing Values - Checked All Columns (Matched)
missing_df = pd.concat([
    pd.DataFrame(train_df.isnull().sum(), columns=['train_miss']),
    pd.DataFrame(test_df.isnull().sum(), columns=['test_miss']),
    pd.DataFrame(new_total_df.isnull().sum(), columns=['total_miss']),
], axis=1).reset_index()
missing_df[(missing_df['train_miss'] > 0) | (missing_df['test_miss'] > 0) | (missing_df['total_miss'] > 0)]

,index,train_miss,test_miss,total_miss
5,monthly_inhand_salary,15002.0,7498.0,0.0
10,type_of_loan,11408.0,5704.0,NaN
12,num_of_delayed_payment,7002.0,3498.0,0.0
14,num_credit_inquiries,1965.0,1035.0,0.0
18,credit_history_age,9030.0,4470.0,NaN
21,amount_invested_monthly,4479.0,2271.0,0.0
23,monthly_balance,1200.0,562.0,0.0
24,credit_score,0.0,NaN,50000.0
25,credit_history_age_in_month,NaN,NaN,16692.0


### 1.1.5. Feature Engineering
After the match, `credit_history_age_in_month` still have missing values.

We could either drop the rows with missing values or impute them.

In [77]:
# Final Convertion
object_columns = new_total_df.select_dtypes(include=['object']).columns.tolist()
for col in object_columns:
    print(f"{col}: {new_total_df[col].unique()}")

customer_id: ['CUS_0xd40' 'CUS_0x21b1' 'CUS_0x2dbc' ... 'CUS_0xaf61' 'CUS_0x8600'
 'CUS_0x942c']
occupation: ['Scientist' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer' 'Lawyer'
 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant' 'Musician'
 'Mechanic' 'Writer' 'Architect']
credit_mix: ['Good' 'Standard' 'Bad']
payment_of_min_amount: ['No' 'NM' 'Yes']
payment_behaviour: ['High_spent_Small_value_payments' 'Low_spent_Large_value_payments'
 'Low_spent_Medium_value_payments' 'Low_spent_Small_value_payments'
 'High_spent_Medium_value_payments' 'High_spent_Large_value_payments']
credit_score: ['Good' 'Standard' 'Poor' nan]


In [78]:
# Convert from categorical to numerical
# 1. customer_id
new_total_df = new_total_df.drop('customer_id', axis=1)

# 2. credit_mix
credit_mix_order = {'Bad':0, 'Standard':1, 'Good':2}
new_total_df['credit_mix'] = pd.Categorical(new_total_df['credit_mix'], categories=credit_mix_order.keys()).codes

# 3. minimum_payment
pmt_map = {'No':0, 'NM':1, 'Yes':2}
new_total_df['payment_of_min_amount'] = new_total_df['payment_of_min_amount'].replace(pmt_map)

# 4. payment_behaviour
new_total_df[['spending_level', 'payment_size']] = new_total_df['payment_behaviour'].str.split('_', n=1, expand=True)
new_total_df['spending_level'] = new_total_df['spending_level'].map({'High':2, 'Low':0, 'Medium':1})
new_total_df['payment_size'] = new_total_df['payment_size'].map({'spent_Large_value_payments':2, 
                                            'spent_Medium_value_payments':1,
                                            'spent_Small_value_payments':0})
new_total_df = new_total_df.drop('payment_behaviour', axis=1)

# 5. credit_score mapping
score_map = {'Poor':0, 'Standard':1, 'Good':2}
new_total_df['credit_score'] = new_total_df['credit_score'].map(score_map)

/tmp/ipykernel_245467/813054238.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_total_df['payment_of_min_amount'] = new_total_df['payment_of_min_amount'].replace(pmt_map)


In [79]:
# Occupation
# We may face Multicolinearity problem if one-hot encoding is applied for occupation.

# # One hot encoding for occupation:
occupation_dummies = pd.get_dummies(new_total_df['occupation'], prefix='occupation')
new_total_df = pd.concat([new_total_df, occupation_dummies], axis=1)
new_total_df = new_total_df.drop('occupation', axis=1)

# LabelEncoding
# le = LabelEncoder()
# new_total_df['occupation'] = le.fit_transform(new_total_df['occupation'])

"""
Instead of one-hot encoding, we can use the following approach:
1. occupation - income level
2. occupation - risk level
3. occupation - stability
"""

# # Occupation - income level, risk level, stability
# income_levels = new_total_df.groupby('occupation')['annual_income'].mean()
# income_quantiles = pd.qcut(income_levels, q=5, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# new_total_df['occupation_income_level'] = new_total_df['occupation'].map(income_quantiles)

# def calculate_risk_score(group):
#     return (
#         group['num_of_delayed_payment'].mean() * 0.3 +
#         group['credit_utilization_ratio'].mean() * 0.3 +
#         (1 - group['payment_of_min_amount'].mean()) * 0.4
#     )
# occupation_risk = new_total_df.groupby('occupation').apply(calculate_risk_score)
# risk_quantiles = pd.qcut(occupation_risk, q=3, labels=['low_risk', 'medium_risk', 'high_risk'])
# new_total_df['occupation_risk_level'] = new_total_df['occupation'].map(risk_quantiles)

# stability_metrics = new_total_df.groupby('occupation').agg({
#     'credit_history_age_in_month': 'mean',
#     'num_bank_accounts': 'mean',
#     'num_credit_card': 'mean'
# }).mean(axis=1)

# stability_quantiles = pd.qcut(stability_metrics, q=3, labels=['low_stability', 'medium_stability', 'high_stability'])
# new_total_df['occupation_stability'] = new_total_df['occupation'].map(stability_quantiles)

# occupation_features = [
#     'occupation_income_level',
#     'occupation_risk_level',
#     'occupation_stability'
# ]

# for feat in occupation_features:
#     new_total_df[feat] = new_total_df[feat].astype('category').cat.codes

# new_total_df = new_total_df.drop('occupation', axis=1)

'\nInstead of one-hot encoding, we can use the following approach:\n1. occupation - income level\n2. occupation - risk level\n3. occupation - stability\n'

In [80]:
new_total_df.columns

Index(['month', 'age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'credit_mix', 'outstanding_debt',
       'credit_utilization_ratio', 'payment_of_min_amount',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_score', 'credit_history_age_in_month', 'auto_loan',
       'credit_builder_loan', 'debt_consolidation_loan', 'home_equity_loan',
       'mortgage_loan', 'not_specified', 'payday_loan', 'personal_loan',
       'student_loan', 'spending_level', 'payment_size',
       'occupation_Accountant', 'occupation_Architect', 'occupation_Developer',
       'occupation_Doctor', 'occupation_Engineer', 'occupation_Entrepreneur',
       'occupation_Journalist', 'occupation_Lawyer', 'occupation_Manager',
       'occupation_Mechanic', 'occupation_Media_Manager',
       'oc

In [81]:
# Standardise
continuous_columns = ['age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']

# Standardise only the continuous columns
scaled_cont_df = pd.DataFrame(StandardScaler().fit_transform(new_total_df[continuous_columns]),
                              columns=continuous_columns,
                              index=new_total_df.index)

# Replace Back
export_df = new_total_df.copy()
export_df[continuous_columns] = scaled_cont_df

# Split train and test
new_train_df = export_df[:train_df_index].dropna()
features = export_df.columns.tolist()
features.remove('credit_score')
new_test_df = export_df[train_df_index:][features].dropna()

# To CSV
new_train_df.to_csv('./data/clean_train.csv')
new_test_df.to_csv('./data/clean_test.csv')

In [82]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = new_train_df.copy()

bool_cols = df.select_dtypes(include='bool').columns
df = df.copy()
df[bool_cols] = df[bool_cols].astype(int)

def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) 
                       for i in range(df.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

# Calculate Value
vif_scaled = calculate_vif(df)
print(vif_scaled)

                        Feature       VIF
3         monthly_inhand_salary  8.913041
18              monthly_balance  6.424523
12                   credit_mix  6.069283
17      amount_invested_monthly  5.159840
7                   num_of_loan  3.999201
39            occupation_Lawyer  3.828305
32        occupation_Accountant  3.690469
36          occupation_Engineer  3.687085
41          occupation_Mechanic  3.672600
42     occupation_Media_Manager  3.661743
34         occupation_Developer  3.650618
38        occupation_Journalist  3.628006
37      occupation_Entrepreneur  3.627970
33         occupation_Architect  3.625642
44         occupation_Scientist  3.622017
45           occupation_Teacher  3.609623
35            occupation_Doctor  3.591524
40           occupation_Manager  3.504048
46            occupation_Writer  3.455047
43          occupation_Musician  3.440076
13             outstanding_debt  2.697045
6                 interest_rate  2.607990
15        payment_of_min_amount  2

## 1.1.6. Imputation (Optional)

In [83]:
new_total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150000 entries, 0 to 49999
Data columns (total 47 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   month                        150000 non-null  int64  
 1   age                          150000 non-null  float64
 2   annual_income                150000 non-null  float64
 3   monthly_inhand_salary        150000 non-null  float64
 4   num_bank_accounts            150000 non-null  float64
 5   num_credit_card              150000 non-null  float64
 6   interest_rate                150000 non-null  float64
 7   num_of_loan                  150000 non-null  float64
 8   delay_from_due_date          150000 non-null  int64  
 9   num_of_delayed_payment       150000 non-null  float64
 10  changed_credit_limit         150000 non-null  float64
 11  num_credit_inquiries         150000 non-null  float64
 12  credit_mix                   150000 non-null  int8   
 13  outst

In [84]:
class FinancialAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(FinancialAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def torch_ae_imputation(data, features_to_impute, epochs=200, batch_size=256):
    # related_features = ['age', 'annual_income', 'num_credit_inquiries',
    #                    'credit_score', 'num_of_loan', 'monthly_inhand_salary',
    #                    'interest_rate', 'num_bank_accounts', 'credit_history_age_in_month']
    related_features = data.columns.tolist()
    related_features.remove('credit_score')
    # Initial imputation
    temp_df = data[related_features].copy()
    imp = SimpleImputer(strategy='median')
    temp_df = pd.DataFrame(imp.fit_transform(temp_df), columns=related_features)
    # Convert to PyTorch tensors
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert to Pytorch Tensors
    continuous_columns = ['age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']
    # Standard only Continuous Columns
    scaler = StandardScaler()
    scaled_cont_df = pd.DataFrame(scaler.fit_transform(temp_df[continuous_columns]),
                                columns=continuous_columns,
                                index=temp_df.index)
    temp_df[continuous_columns] = scaled_cont_df
    tensor_data = torch.FloatTensor(temp_df.values).to(device)
    # Initialize model
    model = FinancialAutoencoder(input_dim=len(related_features)).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # Early stopping Settings
    best_loss = float('inf')
    patience = 10
    no_improve = 0
    # Autoencoder training
    dataset = torch.utils.data.TensorDataset(tensor_data, tensor_data)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # Check if best_model_weights.pth exists, if yes, load the weights, otherwise train the model and save the best weights to best_model_weights.pt
    if os.path.exists('best_impute_model_weights.pth'):
        model.load_state_dict(torch.load('best_impute_model_weights.pth'))
    else:
        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for batch_x, _ in loader:
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_x)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # Print training progress
            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f'Epoch {epoch + 1} / {epochs}, Loss: {train_loss / len(loader)}')
            
            # Early stopping
            with torch.no_grad():
                model.eval()
                val = model(tensor_data)
                val_loss = criterion(val, tensor_data).item()
                if val_loss < best_loss:
                    best_loss = val_loss
                    no_improve = 0
                    best_weights = model.state_dict()
                else:
                    no_improve += 1
                if no_improve >= patience:
                    print(f'Early stopping at epoch {epoch}')
                    break
        # Save the best model weights
        torch.save(best_weights, 'best_impute_model_weights.pth')
    # Model Prediction
    with torch.no_grad():
        reconstructed = model(tensor_data).cpu().numpy()
    reconstructed_df = pd.DataFrame(reconstructed, columns=related_features, index=temp_df.index)
    reconstructed_df[continuous_columns] = scaler.inverse_transform(reconstructed_df[continuous_columns])

    # Replace missing values with the reconstructed values
    for i, feature in enumerate(related_features):
        if feature in features_to_impute:
            # mask = data[feature].isnull()
            # data.loc[mask, feature] = reconstructed_df.loc[mask, feature]
            missing_indices = data[data[feature].isnull()].index
            # Make Sure the missing indices are existed
            valid_indices = missing_indices.intersection(reconstructed_df.index)
            # Replace them
            if not valid_indices.empty:
                data.loc[valid_indices, feature] = reconstructed_df.loc[valid_indices, feature]
    return data

# Check GPU availability
print("Available GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Using CPU")

# Set Random Seed
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Use the function
impute_features = ['credit_history_age_in_month']
new_imputed_total_df = torch_ae_imputation(new_total_df, impute_features, epochs=200)

Available GPU: NVIDIA GeForce RTX 3090 Ti
Epoch 1 / 200, Loss: 0.3400616345246904
Epoch 10 / 200, Loss: 0.1223897112743236
Epoch 20 / 200, Loss: 0.10989641087962498
Epoch 30 / 200, Loss: 0.10511636704765896
Epoch 40 / 200, Loss: 0.10068276744501176
Early stopping at epoch 44


In [85]:
new_imputed_total_df.describe()

,month,age,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,...,credit_builder_loan,debt_consolidation_loan,home_equity_loan,mortgage_loan,not_specified,payday_loan,personal_loan,student_loan,spending_level,payment_size
count,150000.000000,150000.000000,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,...,150000.000000,150000.000000,150000.000000,150000.000000,150000.00000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,5.500000,33.473487,7.058155e+04,4190.326660,5.404053,5.609213,14.542933,3.549293,21.063400,13.186273,...,0.317280,0.310400,0.314000,0.313600,0.31680,0.319440,0.311040,0.310400,0.933440,0.862747
std,3.452064,10.789277,3.664463e+05,3180.874534,2.992640,3.029268,8.784326,2.652622,14.860154,6.324684,...,0.465419,0.462659,0.464118,0.463957,0.46523,0.466261,0.462921,0.462659,0.997786,0.800861
min,0.000000,14.000000,7.005930e+03,303.645417,0.000000,0.000000,1.000000,0.000000,-5.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.750000,25.000000,1.939650e+04,1625.165833,3.000000,4.000000,7.000000,2.000000,10.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.500000,33.000000,3.724022e+04,3089.424167,5.000000,5.000000,13.000000,3.000000,18.000000,14.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,8.250000,42.000000,7.212520e+04,5948.454596,7.000000,7.000000,20.000000,5.000000,28.000000,18.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,2.000000,2.000000
max,11.000000,120.000000,9.976017e+06,15204.633333,100.000000,100.000000,100.000000,100.000000,67.000000,100.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,2.000000,2.000000


In [86]:
# Standardise
continuous_columns = ['age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']

# Standardise only the continuous columns
scaled_cont_df = pd.DataFrame(StandardScaler().fit_transform(new_imputed_total_df[continuous_columns]),
                              columns=continuous_columns,
                              index=new_imputed_total_df.index)

# Replace Back
export_df = new_imputed_total_df.copy()
export_df[continuous_columns] = scaled_cont_df

# Split train and test
new_imputed_train_df = export_df[:train_df_index].dropna()
features = export_df.columns.tolist()
features.remove('credit_score')
new_imputed_test_df = export_df[train_df_index:][features].dropna()

# To CSV
new_imputed_train_df.to_csv('./data/clean_imputed_train.csv')
new_imputed_test_df.to_csv('./data/clean_imputed_test.csv')

In [87]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = new_imputed_train_df.copy()

bool_cols = df.select_dtypes(include='bool').columns
df = df.copy()
df[bool_cols] = df[bool_cols].astype(int)

def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) 
                       for i in range(df.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

# Calculate Value
vif_scaled = calculate_vif(df)
print(vif_scaled)

                        Feature       VIF
3         monthly_inhand_salary  9.083664
18              monthly_balance  6.554644
12                   credit_mix  6.036285
17      amount_invested_monthly  5.199306
7                   num_of_loan  4.208068
39            occupation_Lawyer  3.849224
36          occupation_Engineer  3.727261
33         occupation_Architect  3.719146
42     occupation_Media_Manager  3.714353
32        occupation_Accountant  3.705030
44         occupation_Scientist  3.697840
37      occupation_Entrepreneur  3.691692
34         occupation_Developer  3.683396
41          occupation_Mechanic  3.680352
45           occupation_Teacher  3.650393
38        occupation_Journalist  3.637481
35            occupation_Doctor  3.615252
40           occupation_Manager  3.533655
43          occupation_Musician  3.520287
46            occupation_Writer  3.485803
13             outstanding_debt  2.635862
6                 interest_rate  2.619632
15        payment_of_min_amount  2